In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import nest_asyncio

nest_asyncio.apply()

# Imports

In [ ]:
from typing import Optional

from dotenv import load_dotenv
from IPython.display import Image, display
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import END, START, StateGraph
from langsmith import traceable
from pydantic import BaseModel, Field
from typing_extensions import TypedDict

load_dotenv()

# Vanilla workflow

In [ ]:
class Evaluation(BaseModel):
    explanation: str = Field(
        description="Explain why the text evaluated matches or not the evaluation criteria"
    )
    feedback: str = Field(
        description="Provide feedback to the writer to improve the text"
    )
    is_correct: bool = Field(
        description="Whether the text evaluated matches or not the evaluation criteria"
    )


class State(BaseModel):
    topic: str
    article: Optional[str] = None
    evaluation: Optional[Evaluation] = None


model = ChatOpenAI(model="gpt-4.1-mini")


@traceable
def evaluate_text(state: State) -> Evaluation:
    model_with_str_output = model.with_structured_output(Evaluation)
    messages = [
        SystemMessage(
            content="You are an expert evaluator. Provided with a text, you will evaluate if it's written in British English and if it's appropriate for a young audience. The text must always use British spelling and grammar. Make sure the text doesn't include any em dash."
        ),
        HumanMessage(content=f"Evaluate the following text:\n\n{state.article}"),
    ]
    response = model_with_str_output.invoke(messages)
    return response


@traceable
def fix_text(state: State) -> str:
    messages = [
        SystemMessage(
            content="You are an expert writer. Provided with a text, you will fix the text to improve it."
        ),
        HumanMessage(
            content=f"You were tasked with writing an article about {state.topic}. You wrote the following text:\n\n{state.article}\n\nYou've got the following feedback:\n\n{state.evaluation.feedback}\n\nFix the text to improve it."
        ),
    ]
    response = model.invoke(messages)
    return response.content


@traceable
def generate_text(state: State) -> str:
    messages = [
        SystemMessage(
            content="You are an expert writer. Provided with a topic, you will generate an engaging article with less than 500 words."
        ),
        HumanMessage(content=f"Generate a text about this topic:\n\n{state.topic}"),
    ]
    response = model.invoke(messages)
    return response.content


@traceable
def generate_text_dispatch(state: State) -> str:
    if state.evaluation:
        return fix_text(state)
    return generate_text(state)


@traceable
def run_workflow(topic: str) -> State:
    state = State(topic=topic)

    for _ in range(4):
        state.article = generate_text_dispatch(state)
        state.evaluation = evaluate_text(state)
        if state.evaluation.is_correct:
            return state

    return state


state = run_workflow("Substance abuse of athletes")

# LangGraph implementation

In [ ]:
class Evaluation(BaseModel):
    explanation: str
    feedback: str
    is_correct: bool


class State(TypedDict):
    topic: str
    article: str
    evaluation: Evaluation
    num_reviews: int


model = ChatOpenAI(model="gpt-4.1-mini", temperature=1)
model_evaluator = ChatOpenAI(model="gpt-4.1-mini", temperature=0)


def generate_article(state: State) -> dict:
    messages = [
        SystemMessage(
            content="You are an expert writer. Provided with a topic, you will generate an engaging article with less than 500 words."
        ),
        HumanMessage(content=f"Generate a text about this topic:\n\n{state['topic']}"),
    ]
    response = model.invoke(messages)
    return {"article": response.content}


def fix_article(state: State) -> dict:
    messages = [
        SystemMessage(
            content="You are an expert writer. Provided with a text, you will fix the text to improve it. The text must always use British spelling and grammar."
        ),
        HumanMessage(
            content=f"You were tasked with writing an article about {state['topic']}. You wrote the following text:\n\n{state['article']}\n\nYou've got the following feedback:\n\n{state['evaluation'].feedback}\n\nFix the text to improve it."
        ),
    ]
    response = model.invoke(messages)
    return {"article": response.content}


def evaluate_article(state: State) -> dict:
    model_with_str_output = model_evaluator.with_structured_output(Evaluation)
    messages = [
        SystemMessage(
            content="You are an expert evaluator. Provided with a text, you will evaluate if it's written in British English and if it's appropriate for a young audience. The text must always use British spelling and grammar. Make sure the text doesn't include any em dash. Be very strict with the evaluation. In case of doubt, return a negative evaluation."
        ),
        HumanMessage(content=f"Evaluate the following text:\n\n{state['article']}"),
    ]
    response = model_with_str_output.invoke(messages)
    return {"evaluation": response, "num_reviews": state.get("num_reviews", 0) + 1}


def route_text(state: State) -> str:
    evaluation = state.get("evaluation", None)
    num_reviews = state.get("num_reviews", 0)
    if evaluation and not evaluation.is_correct and num_reviews < 3:
        return "Fail"
    return "Pass"


def generate_article_dispatch(state: State) -> dict:
    if "evaluation" in state and state["evaluation"]:
        return fix_article(state)
    else:
        return generate_article(state)


workflow = StateGraph(State)

workflow.add_node("generate_article", generate_article_dispatch)
workflow.add_node("evaluate_article", evaluate_article)

workflow.add_edge(START, "generate_article")
workflow.add_edge("generate_article", "evaluate_article")
workflow.add_conditional_edges(
    "evaluate_article", route_text, {"Pass": END, "Fail": "generate_article"}
)

chain = workflow.compile()

display(Image(chain.get_graph().draw_mermaid_png()))

In [ ]:
state = chain.invoke({"topic": "Artificial Intelligence"})

print("Article:")
print(state["article"])
print("\n--- --- ---\n")

if "evaluation" in state and state["evaluation"] is not None:
    print("Evaluation:")
    print(state["evaluation"])
    print("\n--- --- ---\n")

    if "article" in state:
        print("Article:")
        print(state["article"])
else:
    print("Article passed quality gate - no revised content detected!")

## Exercise:

Transform the prompt chain workflow that generates recipes into a evaluator optimizer workflow. It should make sure that the recipe is accurate, easy to follow, and that it has few ingredients.

In [ ]:
class Ingredients(BaseModel):
    ingredients: list[str]


class RecipeEvaluation(BaseModel):
    score: int = Field(
        description="The score of the recipe between 1 and 5. 1 is the worst and 5 is the best."
    )
    feedback: str = Field(description="The feedback on the recipe.")


class State(TypedDict):
    recipe_request: str
    recipe_content: str
    ingredients: list[str]
    num_revisions: int
    evaluation: RecipeEvaluation | None


def get_ingredients(state: State) -> dict:
    model_with_structure = model.with_structured_output(Ingredients)
    messages = [
        SystemMessage(
            content="You are an expert chef. Provided with a recipe request, you will generate a list of ingredients."
        ),
        HumanMessage(
            content=f"Generate a list of ingredients for the following recipe request: {state['recipe_request']}"
        ),
    ]
    return {"ingredients": model_with_structure.invoke(messages).ingredients}


def generate_recipe_content(state: State) -> str:
    messages = [
        SystemMessage(
            content="You are an expert chef. Provided with a recipe request and a list of ingredients, you will generate a recipe. You should return the recipe content only, no other text."
        ),
        HumanMessage(
            content=f"Write a recipe for {state['recipe_request']} with the following ingredients:\n\n{state['ingredients']}"
        ),
    ]
    return {"recipe_content": model.invoke(messages).content}


def fix_recipe(state: State) -> str:
    messages = [
        SystemMessage(
            content="You are an expert chef. Provided with a recipe and feedback, you should try to fix the recipe. You should only return the fixed recipe, no other text."
        ),
        HumanMessage(
            content=f"Fix the following recipe:\n\n{state['recipe_content']}\n\nFeedback: {state['evaluation'].feedback}"
        ),
    ]
    return {
        "recipe_content": model.invoke(messages).content,
        "num_revisions": state.get("num_revisions", 0) + 1,
    }


def recipe_dispatch(state: State) -> str:
    if "evaluation" in state and state["evaluation"]:
        return fix_recipe(state)
    else:
        return generate_recipe_content(state)


def route_recipe(state: State) -> str:
    evaluation = state.get("evaluation", None)
    num_revisions = state.get("num_revisions", 0)
    if evaluation and evaluation.score < 3 and num_revisions < 3:
        return "Fail"
    return "Pass"


def evaluate_recipe(state: State) -> str:
    model_with_structure = model.with_structured_output(RecipeEvaluation)
    messages = [
        SystemMessage(
            content="You are an expert chef. Provided with a recipe request, a list of ingredients and a recipe, you should evaluate the recipe and return a score between 1 and 5. 1 is the worst and 5 is the best. You should evaluate on this criteria: \n\n- The recipe is easy to follow to beginners\n- The recipe MUST be healthy, low in calories, and low in sugar\n- The recipe uses very few ingredients (be very strict)"
        ),
        HumanMessage(
            content=f"Evaluate the following recipe for {state['recipe_request']} with the following ingredients:\n\n{state['ingredients']}\n\nand the following recipe:\n\n{state['recipe_content']}"
        ),
    ]
    response = model_with_structure.invoke(messages)
    return {"evaluation": response}


workflow = StateGraph(State)

workflow.add_node("get_ingredients", get_ingredients)
workflow.add_node("generate_recipe_content", recipe_dispatch)
workflow.add_node("evaluate", evaluate_recipe)

workflow.add_edge(START, "get_ingredients")
workflow.add_edge("get_ingredients", "generate_recipe_content")
workflow.add_edge("generate_recipe_content", "evaluate")
workflow.add_conditional_edges(
    source="evaluate",
    path=route_recipe,
    path_map={"Fail": "generate_recipe_content", "Pass": END},
)

chain = workflow.compile()

display(Image(chain.get_graph().draw_mermaid_png()))

In [ ]:
state = chain.invoke({"recipe_request": "a recipe for a cake"})